# Padrões Arquiteturais de Software - Engenharia de Software II

In [ ]:
# Célula 1: Instalação de Dependências
# Descomente e execute se necessário.
# 'radon' é para a análise de dependências/complexidade em Python.
# Troque 'radon' por 'madge' se for um projeto JS/TS.

!pip install openai huggingface_hub sentence-transformers scikit-learn numpy
!pip install radon

print("Bibliotecas prontas. (Execute !pip install ... se necessário)")

Bibliotecas prontas. (Execute !pip install ... se necessário)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
# Célula 2: Imports e Configuração Inicial
import os
import subprocess
import json
import shutil
from openai import OpenAI
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import numpy as np
import warnings
from radon.cli import CCHarvester # Para análise de complexidade/dependência Python

# Suprimir avisos
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")

# --- 1. CONFIGURAÇÃO DA API HUGGING FACE ---
# !! IMPORTANTE !! Substitua pela sua API Key.
HF_API_KEY = "SUA_API_KEY_AQUI"

if HF_API_KEY == "SUA_API_KEY_AQUI":
    print("="*50)
    print("!! ALERTA: Por favor, insira sua API Key do Hugging Face na Célula 2 !!")
    print("="*50)

client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=HF_API_KEY,
)

# --- 2. CONFIGURAÇÃO DO MODELO ---
# Escolha um modelo de chat disponível no Hugging Face
# "meta-llama/Llama-3.1-8B-Instruct",
# "deepseek-ai/DeepSeek-V3.2-Exp",
# "zai-org/GLM-4.6",
MODELO_HF = "Qwen/Qwen2.5-7B-Instruct"

# --- 3. CONFIGURAÇÃO DO REPOSITÓRIO (!! IMPORTANTE !!) ---
# !! MUDE AQUI !!
REPO_URL = "https://github.com/unclecode/crawl4ai.git" # Exemplo: FastAPI (Python)
REPO_PATH = "repositorio_clonado"

# !! MUDE AQUI !!
# Filtro de linguagem para o Passo 4 (Embedding)
CODE_FILE_EXTENSION = ".py"
# Filtro de pastas a ignorar (otimiza a análise)
IGNORE_DIRS = {'node_modules', '.git', '.vscode', '__pycache__', 'venv', 'docs', 'tests', 'examples'}

# --- 4. FUNÇÃO HELPER PARA CHAMAR A LLM ---
#       Esta função irá determinar os papéis da LLM e informa-la do prompt
#       em questão, depois retornar a resposta ao mesmo. Tudo isso com error handling
def analisar_com_llm(prompt_sistema, prompt_usuario):
    """
    Função auxiliar para fazer chamadas à API do Hugging Face
    usando a interface do OpenAI.
    """
    print("--- 🤖 Chamando LLM para análise... ---")
    # if HF_API_KEY == "SUA_API_KEY_AQUI":
    #     print("!! ERRO: API Key não configurada. Pulando chamada... !!")
    #     return "(API Key não configurada)"

    try:
        completion = client.chat.completions.create(
            model=MODELO_HF,
            messages=[
                {"role": "system", "content": prompt_sistema},
                {"role": "user", "content": prompt_usuario},
            ],
            temperature=0.1,
        )
        resposta = completion.choices[0].message.content
        print("--- ✅ Análise da LLM recebida. ---")
        return resposta
    except Exception as e:
        print(f"--- ❌ Erro ao chamar a LLM: {e} ---")
        return f"(Erro na chamada da API: {e})"

print("Configuração inicial concluída.")

In [ ]:
# Célula 3: Clonando o Repositório Real
print("\n--- INICIANDO CLONE DO REPOSITÓRIO ---")

# Se já houver um repositório nesse caminho, ele é deletado
if os.path.exists(REPO_PATH):
    print(f"Removendo diretório existente: {REPO_PATH}")
    shutil.rmtree(REPO_PATH)

print(f"Clonando {REPO_URL} para {REPO_PATH}...")
try:
    # Usamos --depth 1 para um clone rápido (shallow clone)
    # Remova --depth 1 se a análise completa do Git Log (Passo 5) for crucial
    subprocess.run(
        ["git", "clone", REPO_URL, REPO_PATH],
        check=True, capture_output=True, text=True
    )
    print("Repositório clonado com sucesso.")
except FileNotFoundError:
    print("ERRO: 'git' não encontrado. Por favor, instale o Git.")
except subprocess.CalledProcessError as e:
    print(f"ERRO ao clonar o repositório: {e.stderr}")
except Exception as e:
    print(f"Um erro inesperado ocorreu: {e}")

In [ ]:
# Célula 4: PASSO 1 - Análise da Documentação
print("\n--- INICIANDO PASSO 1: Análise da Documentação ---")

documentacao_completa_para_chunking = "" # Coleta toda a documentação relevante para chunking posterior
doc_files = []
# Limita a busca para evitar arquivos .md em 'node_modules' etc.
for root, dirs, files in os.walk(REPO_PATH):
    # Remove diretórios ignorados da busca
    dirs[:] = [d for d in dirs if d not in IGNORE_DIRS]

    for file in files:
        if file.lower().endswith(".md"):
            filepath = os.path.join(root, file)
            doc_files.append(filepath)
            try:
                with open(filepath, "r", encoding="utf-8") as f:
                    conteudo = f.read() # Lê o conteúdo completo para chunking posterior
                    # "os.path.relpath" retorna o caminho do conteudo, e "conteudo" o conteudo em si
                    documentacao_completa_para_chunking += f"--- CONTEÚDO DE {os.path.relpath(filepath, REPO_PATH)} ---\n{conteudo}\n\n"
            except Exception as e:
                print(f"Erro ao ler {filepath}: {e}")

print(f"Arquivos .md encontrados e lidos: {len(doc_files)}")

# --- Chunking da documentação para LLM ---
# Estimativa conservadora de chars por token para o modelo (varia por modelo)
CHARS_PER_TOKEN_ESTIMATE = 3.5
MODEL_MAX_TOKENS = 16384
PROMPT_OVERHEAD_TOKENS = 500 # Estimativa para o prompt do sistema + partes fixas do prompt do usuário

MAX_DOCUMENT_TOKENS = MODEL_MAX_TOKENS - PROMPT_OVERHEAD_TOKENS
MAX_CHUNK_CHARS = int(MAX_DOCUMENT_TOKENS * CHARS_PER_TOKEN_ESTIMATE)

documentacao_chunks = []
if len(documentacao_completa_para_chunking) > MAX_CHUNK_CHARS:
    for i in range(0, len(documentacao_completa_para_chunking), MAX_CHUNK_CHARS):
        documentacao_chunks.append(documentacao_completa_para_chunking[i:i + MAX_CHUNK_CHARS])
    print(f"Documentação dividida em {len(documentacao_chunks)} chunks para análise.")
else:
    documentacao_chunks.append(documentacao_completa_para_chunking)
    print("Documentação pequena o suficiente para uma única análise.")


prompt_sistema = "Você é um engenheiro de software sênior. Sua tarefa é analisar a documentação de um projeto e identificar padrões arquiteturais e de design."
analises_parciais = []

for i, chunk in enumerate(documentacao_chunks):
    prompt_usuario_chunk = f"""
    Com base na seguinte parte da documentação (parte {i+1} de {len(documentacao_chunks)}), identifique:
    1. Quais padrões arquiteturais (ex: MVC, Layered, Microserviços) são explicitamente mencionados?
    2. Quais princípios de design (ex: separação de responsabilidades, injeção de dependência) são descritos?
    3. Faça um resumo da arquitetura proposta por esta parte da documentação.

    --- INÍCIO DO CHUNK {i+1} ---
    {chunk}
    --- FIM DO CHUNK {i+1} ---
    """
    print(f"\n--- Analisando chunk {i+1} de {len(documentacao_chunks)} ---")
    analise_chunk = analisar_com_llm(prompt_sistema, prompt_usuario_chunk)
    analises_parciais.append(f"Análise do Chunk {i+1}:\n{analise_chunk}\n")

# --- Síntese das análises parciais ---
if len(analises_parciais) > 1:
    synthesis_prompt_sistema = "Você é um engenheiro de software sênior. Sua tarefa é sintetizar múltiplas análises parciais de documentação em um relatório coeso e abrangente, focado em padrões arquiteturais e de design."
    synthesis_prompt_usuario = f"""
    Sintetize as seguintes análises parciais de documentação em um relatório único e coeso sobre os padrões arquiteturais e de design do projeto. Evite repetições e foque nos pontos principais de cada análise. Priorize a identificação dos padrões mais relevantes e a criação de um resumo claro da arquitetura.

    {'\n\n'.join(analises_parciais)}
    """
    print("\n--- INICIANDO SÍNTESE DAS ANÁLISES PARCIAIS ---")

    # Ajusta o prompt de síntese caso ele também exceda o limite de tokens
    if len(synthesis_prompt_usuario) / CHARS_PER_TOKEN_ESTIMATE > MODEL_MAX_TOKENS:
        print("!! ALERTA: O prompt de síntese é muito longo e será truncado para caber no limite de tokens. !!")
        synthesis_prompt_usuario = synthesis_prompt_usuario[:int(MODEL_MAX_TOKENS * CHARS_PER_TOKEN_ESTIMATE * 0.9)] + "\n\n[... O prompt de síntese foi truncado para caber no limite de tokens ...]"


    analise_final = analisar_com_llm(synthesis_prompt_sistema, synthesis_prompt_usuario)
    print("\n--- SÍNTESE CONCLUÍDA ---")
else:
    analise_final = analises_parciais[0] if analises_parciais else "Nenhuma documentação para analisar."

# Salvando o resultado em um arquivo markdown
output_filename = "analise_passo1.md"
with open(output_filename, "w", encoding="utf-8") as f:
    f.write("=== RESULTADO PASSO 1 (LLM) ===\n")
    f.write(analise_final)
    f.write("\n-----------------------------------")
print(f"Resultado do Passo 1 salvo em '{output_filename}'")

In [ ]:
cat analise_passo1.md

In [ ]:
# Célula 5: PASSO 2 - Análise da Estrutura de Pastas
print("\n--- INICIANDO PASSO 2: Análise da Estrutura de Pastas ---")

# Usando 'tree' (Linux/macOS) ou 'ls -R' (alternativa)
estrutura_pastas = ""
try:
    # Tenta usar 'tree' com profundidade 3 e ignorando diretórios
    ignore_str = "|".join(IGNORE_DIRS)
    comando_tree = ["tree", "-L", "3", "-I", ignore_str, REPO_PATH]
    processo = subprocess.run(comando_tree, capture_output=True, text=True, check=True, encoding="utf-8")
    estrutura_pastas = processo.stdout
except (FileNotFoundError, subprocess.CalledProcessError):
    print("Comando 'tree' não encontrado ou falhou. Gerando lista simples com pastas e arquivos.")
    # Alternativa mais simples: percorrer diretórios e arquivos
    lista_itens = []
    for root, dirs, files in os.walk(REPO_PATH, topdown=True):
        # Remove diretórios ignorados da busca
        dirs[:] = [d for d in dirs if d not in IGNORE_DIRS]

        # Limita a profundidade
        level = root.replace(REPO_PATH, '').count(os.sep)
        if level > 3:
            continue

        indent = " " * 4 * level
        lista_itens.append(f"{indent}{os.path.basename(root)}/")
        for f in files:
            lista_itens.append(f"{indent}    {f}") # Adiciona arquivos com indentação extra

    estrutura_pastas = "\n".join(lista_itens)

print("\nEstrutura de Pastas e Arquivos Gerada (Nível 3):")
print(estrutura_pastas)

# Preparando prompts para a LLM
prompt_sistema = "Você é um arquiteto de software. Analise a estrutura de arquivos de um projeto para inferir seu padrão arquitetural."
prompt_usuario = f"""
Dada a seguinte estrutura de arquivos e pastas (Nível 3):

{estrutura_pastas}

1. Qual padrão arquitetural esta estrutura sugere? (ex: Layered, baseada em features/domínio, monorepo, etc.)
2. Justifique sua resposta com base nos nomes das pastas e arquivos.
"""

# Chamando a LLM
analise_passo2 = analisar_com_llm(prompt_sistema, prompt_usuario)
output_filename = "analise_passo2.md"
with open(output_filename, "w", encoding="utf-8") as f:
    f.write("=== RESULTADO PASSO 2 (LLM) ===\n")
    f.write(analise_passo2)
    f.write("\n-----------------------------------")
print(f"Resultado do Passo 2 salvo em '{output_filename}'")

In [ ]:
cat analise_passo2.md

In [ ]:
# Célula 6: PASSO 3 - Análise do Grafo de Dependências (Exemplo com Python/radon)
print("\n--- INICIANDO PASSO 3: Análise do Grafo de Dependências ---")
print(f"Usando 'radon' para analisar código {CODE_FILE_EXTENSION} (Python).")
print("Este passo precisa ser adaptado para outras linguagens (ex: 'madge' para JS/TS).")

# Coletar arquivos de código
arquivos_para_analise = []
for root, dirs, files in os.walk(REPO_PATH):
    dirs[:] = [d for d in dirs if d not in IGNORE_DIRS]
    for file in files:
        if file.endswith(CODE_FILE_EXTENSION):
            arquivos_para_analise.append(os.path.join(root, file))

grafo_dependencias = {}

if CODE_FILE_EXTENSION == ".py" and arquivos_para_analise:
    print(f"Analisando {len(arquivos_para_analise)} arquivos Python com 'radon'...")
    try:
        # Usamos CCHarvester do radon para obter o grafo de chamadas/dependências
        # Nota: Isso é complexo. Uma análise de 'import' seria mais simples.
        # Vamos simplificar e focar nos 'imports'

        # Análise de Imports (Simplificada)
        for filepath in arquivos_para_analise[:50]: # Limita a 50 arquivos para velocidade
            rel_path = os.path.relpath(filepath, REPO_PATH)
            grafo_dependencias[rel_path] = []
            with open(filepath, "r", encoding="utf-8") as f:
                for line in f:
                    if line.strip().startswith("import ") or line.strip().startswith("from "):
                        # Lógica muito crua - apenas para demonstrar
                        parts = line.split()
                        if len(parts) > 1:
                             # Captura 'from fastapi' ou 'import uvicorn'
                            if parts[0] == "from":
                                grafo_dependencias[rel_path].append(parts[1])
                            elif parts[0] == "import":
                                grafo_dependencias[rel_path].append(parts[1])

        output_grafo_simulado = json.dumps(grafo_dependencias, indent=2)
        print("Grafo de dependências (amostra de imports) gerado.")

    except Exception as e:
        output_grafo_simulado = f"Erro ao analisar dependências com radon: {e}"
        print(output_grafo_simulado)
else:
    output_grafo_simulado = "Linguagem não suportada por este script ou nenhum arquivo encontrado."
    print(output_grafo_simulado)


# Preparando prompts para a LLM
prompt_sistema = "Você é um analista de arquitetura de software especializado em grafos de dependência."
prompt_usuario = f"""
Analise o seguinte grafo de dependências (amostrado e simplificado, mostrando 'imports' de arquivos):

{output_grafo_simulado}

1. Com base nesta amostra, o projeto parece ter alta ou baixa acoplamento?
2. Você consegue identificar dependências principais (bibliotecas externas) que definem a arquitetura? (ex: 'fastapi', 'django', 'react')
3. Você consegue identificar 'módulos' ou 'camadas' com base nos imports internos (se houver)?
"""

# Chamando a LLM
analise_passo3 = analisar_com_llm(prompt_sistema, prompt_usuario)
output_filename = "analise_passo3.md"
with open(output_filename, "w", encoding="utf-8") as f:
    f.write("=== RESULTADO PASSO 3 (LLM) ===\n")
    f.write(analise_passo3)
    f.write("\n-----------------------------------")
print(f"Resultado do Passo 3 salvo em '{output_filename}'")

In [ ]:
cat analise_passo3.md

In [ ]:
# Célula 7: PASSO 4 - Embedding e Clusterização do Código
print("\n--- INICIANDO PASSO 4: Embedding e Clusterização ---")

# --- 4.1. Coletar Arquivos de Código ---
arquivos_codigo = []
for root, dirs, files in os.walk(REPO_PATH):
    dirs[:] = [d for d in dirs if d not in IGNORE_DIRS]
    for file in files:
        if file.endswith(CODE_FILE_EXTENSION):
            filepath = os.path.join(root, file)
            try:
                with open(filepath, "r", encoding="utf-8") as f:
                    conteudo = f.read()
                # Usamos o caminho relativo para melhor interpretação
                arquivos_codigo.append((os.path.relpath(filepath, REPO_PATH), conteudo))
            except Exception as e:
                print(f"Erro ao ler {filepath}: {e}, pulando...")

print(f"Encontrados {len(arquivos_codigo)} arquivos '{CODE_FILE_EXTENSION}' para embedding.")

# Limitar a 1000 arquivos para evitar estouro de memória/tempo
if len(arquivos_codigo) > 1000:
    print("Repositório muito grande. Amostrando 1000 arquivos aleatoriamente.")
    indices = np.random.choice(len(arquivos_codigo), 1000, replace=False)
    arquivos_codigo = [arquivos_codigo[i] for i in indices]

if not arquivos_codigo:
    print("Nenhum arquivo de código encontrado. Pulando Passo 4.")
    analise_passo4 = "Nenhum arquivo de código encontrado para clusterizar."
else:
    # --- 4.2. Gerar Embeddings ---
    print("Carregando modelo de embedding (sentence-transformers)...")
    model = SentenceTransformer('all-MiniLM-L6-v2')

    conteudos = [conteudo for _, conteudo in arquivos_codigo]
    embeddings = model.encode(conteudos)
    print(f"Embeddings gerados. Shape: {embeddings.shape}")

    # --- 4.3. Clusterização (K-Means) ---
    # Definimos 6 clusters como um chute inicial razoável (ex: models, controllers, services, utils, config, tests)
    num_clusters = min(6, len(arquivos_codigo)) # Não ter mais clusters que arquivos
    kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
    labels = kmeans.fit_predict(embeddings)

    clusters = {}
    for i in range(len(arquivos_codigo)):
        label = labels[i]
        if label not in clusters:
            clusters[label] = []
        clusters[label].append(arquivos_codigo[i][0]) # Adiciona o nome do arquivo

    print("\nArquivos agrupados por cluster (amostra):")
    for label, files in clusters.items():
        print(f"Cluster {label}: {files[:5]}... ({len(files)} arquivos)")

    # --- 4.4. Analisar Clusters com a LLM ---
    print("\nAnalisando o 'propósito' de cada cluster com a LLM...")
    analise_clusters = {}
    prompt_sistema_cluster = f"Você é um programador sênior analisando código {CODE_FILE_EXTENSION}. Sua tarefa é dar um rótulo funcional a um cluster de arquivos (ex: 'Controladores HTTP', 'Lógica de Banco de Dados', 'Funções Utilitárias', 'Configuração')."

    for label, files in clusters.items():
        # Pegar o conteúdo do primeiro arquivo do cluster como amostra
        sample_filepath_rel = files[0]
        sample_filepath_abs = os.path.join(REPO_PATH, sample_filepath_rel)

        with open(sample_filepath_abs, "r", encoding="utf-8") as f:
            sample_content = f.read(2000) # Limita a 2000 caracteres

        prompt_usuario = f"""
        Estou analisando o **Cluster {label}**.
        Um arquivo de amostra deste cluster é `{sample_filepath_rel}`.
        Seu conteúdo (truncado) é:
        ```{CODE_FILE_EXTENSION}
        {sample_content}
        Com base nesta amostra, qual é a provável responsabilidade principal (o rótulo funcional) dos arquivos neste cluster?
        """

        rotulo_cluster = analisar_com_llm(prompt_sistema_cluster, prompt_usuario)
        analise_clusters[f"Cluster {label}"] = {
            "amostra_arquivos": files[:5],
            "total_arquivos": len(files),
            "rotulo_llm": rotulo_cluster
        }

print("\n=== RESULTADO PASSO 4 (LLM) ===")
analise_passo4 = json.dumps(analise_clusters, indent=2)
output_filename = "analise_passo4.md"
with open(output_filename, "w", encoding="utf-8") as f:
    f.write("=== RESULTADO PASSO 4 (LLM) ===\n")
    f.write(analise_passo4)
    f.write("\n-----------------------------------")
print(f"Resultado do Passo 4 salvo em '{output_filename}'")

In [ ]:
cat analise_passo4.md

In [ ]:
# Célula 8: PASSO 5 - Análise do Git Log
print("\n--- INICIANDO PASSO 5: Análise do Git Log (Filtrado) ---")
print("Nota: Este passo pode falhar se você usou --depth 1 no clone.")

git_log_filtrado = ""
try:
    # Filtramos por palavras-chave arquiteturais
    comando = [
        "git", "log", "--all", "--max-count=100", # Limita aos últimos 100 commits
        "--grep=refactor",
        "--grep=architecture",
        "--grep=pattern",
        "--grep=layer",
        "-i" # Ignora case
    ]
    processo = subprocess.run(comando, cwd=REPO_PATH, capture_output=True, text=True, check=True, encoding="utf-8")
    git_log_filtrado = processo.stdout

    if not git_log_filtrado:
        git_log_filtrado = "(Nenhum commit relevante encontrado com os filtros nos últimos 100 commits)"

except subprocess.CalledProcessError as e:
    # Se o clone foi --depth 1, o 'git log' pode não ter histórico
    if "shallow" in e.stderr:
        git_log_filtrado = "Erro: O repositório foi clonado com '--depth 1' (shallow clone). O histórico de commits completo não está disponível para análise. Remova '--depth 1' da Célula 3 para analisar o log."
    else:
        git_log_filtrado = f"Erro ao rodar git log: {e.stderr}"
except Exception as e:
    git_log_filtrado = f"Erro ao rodar git log: {e}"


print("Log filtrado (commits com 'refactor', 'architecture', etc.):")
print(git_log_filtrado[:1000] + "...") # Mostra apenas o início

# --- Chunking do Git Log para LLM ---
# Reutiliza as constantes de chunking da Célula 4
git_log_chunks = []
if len(git_log_filtrado) > MAX_CHUNK_CHARS:
    for i in range(0, len(git_log_filtrado), MAX_CHUNK_CHARS):
        git_log_chunks.append(git_log_filtrado[i:i + MAX_CHUNK_CHARS])
    print(f"Git Log dividido em {len(git_log_chunks)} chunks para análise.")
else:
    git_log_chunks.append(git_log_filtrado)
    print("Git Log pequeno o suficiente para uma única análise.")


prompt_sistema = "Você é um gerente de engenharia analisando o histórico de commits de um projeto para encontrar decisões arquiteturais."
analises_parciais_git = []

for i, chunk in enumerate(git_log_chunks):
    prompt_usuario_chunk_git = f"""
    Analise APENAS a seguinte parte do histórico de commits (parte {i+1} de {len(git_log_chunks)}).
    O que essas mensagens de commit revelam sobre a evolução ou a intenção da arquitetura do software?
    (Se nenhuma mensagem for encontrada ou um erro for reportado, apenas declare isso).

    --- INÍCIO DO CHUNK {i+1} ---
    {chunk}
    --- FIM DO CHUNK {i+1} ---
    """
    print(f"\n--- Analisando chunk {i+1} de {len(git_log_chunks)} do Git Log ---")
    analise_chunk_git = analisar_com_llm(prompt_sistema, prompt_usuario_chunk_git)
    analises_parciais_git.append(f"Análise do Chunk {i+1} do Git Log:\n{analise_chunk_git}\n")

# --- Síntese das análises parciais do Git Log ---
if len(analises_parciais_git) > 1:
    synthesis_prompt_sistema_git = "Você é um gerente de engenharia. Sua tarefa é sintetizar múltiplas análises parciais de históricos de commits em um relatório coeso, focado em decisões e evolução arquitetural."
    synthesis_prompt_usuario_git = f"""
    Sintetize as seguintes análises parciais do Git Log em um relatório único e coeso sobre as decisões e a evolução arquitetural do projeto. Evite repetições e foque nos pontos principais de cada análise.

    {'\n\n'.join(analises_parciais_git)}
    """
    print("\n--- INICIANDO SÍNTESE DAS ANÁLISES PARCIAIS DO GIT LOG ---")

    # Ajusta o prompt de síntese caso ele também exceda o limite de tokens
    if len(synthesis_prompt_usuario_git) / CHARS_PER_TOKEN_ESTIMATE > MODEL_MAX_TOKENS:
        print("!! ALERTA: O prompt de síntese do Git Log é muito longo e será truncado para caber no limite de tokens. !!")
        synthesis_prompt_usuario_git = synthesis_prompt_usuario_git[:int(MODEL_MAX_TOKENS * CHARS_PER_TOKEN_ESTIMATE * 0.9)] + "\n\n[... O prompt de síntese foi truncado para caber no limite de tokens ...]"

    analise_final_git = analisar_com_llm(synthesis_prompt_sistema_git, synthesis_prompt_usuario_git)
    print("\n--- SÍNTESE DO GIT LOG CONCLUÍDA ---")
else:
    analise_final_git = analises_parciais_git[0] if analises_parciais_git else "Nenhum Git Log para analisar."


# Salvando o resultado em um arquivo markdown
output_filename = "analise_passo5.md"
with open(output_filename, "w", encoding="utf-8") as f:
    f.write("=== RESULTADO PASSO 5 (LLM) ===\n")
    f.write(analise_final_git)
    f.write("\n-----------------------------------")
print(f"Resultado do Passo 5 salvo em '{output_filename}'")

In [ ]:
cat analise_passo5.md

In [ ]:
# Célula 9: PASSO 6 - Análise Final (Síntese)
print("\n--- INICIANDO PASSO 6: Síntese Final ---")
print("Combinando todas as análises para o relatório final...")

# Preparando o prompt final com todas as evidências
prompt_sistema_final = """
Você é um Arquiteto de Software Sênior e está escrevendo o relatório final de uma análise de repositório.
Você recebeu 5 blocos de evidências. Sua tarefa é sintetizar TODAS elas em uma conclusão coesa e justificada.
Seja direto e use as evidências para apoiar sua conclusão.
"""

prompt_usuario_final = f"""
Por favor, gere um relatório de análise arquitetural final.
Combine as seguintes 5 fontes de evidência para chegar a uma conclusão sobre os padrões arquiteturais de engenharia de software deste projeto.

--- EVIDÊNCIA 1: Análise da Documentação ---
{analise_passo1}

--- EVIDÊNCIA 2: Análise da Estrutura de Pastas ---
{analise_passo2}

--- EVIDÊNCIA 3: Análise do Grafo de Dependências ---
{analise_passo3}

--- EVIDÊNCIA 4: Análise da Clusterização de Código ---
{analise_passo4}

--- EVIDÊNCIA 5: Análise do Git Log ---
{analise_passo5}

--- RELATÓRIO FINAL ---
Com base em TUDO acima, responda:
1. Qual é o padrão arquitetural primário identificado?
2. As evidências são consistentes ou contraditórias? (Ex: A documentação bate com o código? A estrutura de pastas reflete os clusters?)
3. Forneça uma justificativa resumida para sua conclusão, citando pelo menos 3 das fontes de evidência.
"""

# Chamando a LLM para o relatório final
print("\nGerando relatório final...")
relatorio_final = analisar_com_llm(prompt_sistema_final, prompt_usuario_final)
output_filename = "analise_passo6.md"
with open(output_filename, "w", encoding="utf-8") as f:
    f.write("=== RESULTADO PASSO 6 (LLM) ===\n")
    f.write(relatorio_final)
    f.write("\n-----------------------------------")
print(f"Resultado do Passo 6 salvo em '{output_filename}'")

In [ ]:
cat analise_passo6.md